In [135]:
import cv2
from PIL import Image
import numpy as np

In [206]:
video_capture = cv2.VideoCapture('C:\\Users\\ASMITPAUL\\Downloads\\volleyball_match.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('C:\\Users\\ASMITPAUL\\Downloads\\volleyball_output.mp4',fourcc, 22.0, (1280,720))

volleyball_cascade = cv2.CascadeClassifier('volleyball.xml')

while True:
    players_yellow = 0
    players_red = 0
    ret, frame = video_capture.read()
    if not ret:
        break

    # VOLLEY BALL DETECTION
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    volleyballs = volleyball_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), maxSize=(100,100))
    for (x, y, w, h) in volleyballs:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # YELLO SHIRT DETECTION
    
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_limit = np.array([10, 50, 50])
    upper_limit = np.array([30, 255, 255])

    yellow_mask = cv2.inRange(hsv_image, lower_limit, upper_limit)

    contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)

        if area > 500 and area < 10000:
            players_yellow += 1
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

    # RED SHIRT DETECTION
    
    lower_limit = np.array([150, 200, 170])
    upper_limit = np.array([255, 255, 200])

    red_mask = cv2.inRange(hsv_image, lower_limit, upper_limit)

    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)

        if area > 300 and area < 7000:
            players_red += 1
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 255), 2)

    frame = cv2.putText(frame, f"Red Players: {players_red}", (10,70), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 2)
    frame = cv2.putText(frame, f"Yellow Players: {players_yellow}", (10,110), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 2)

    if ret == True:
        out.write(frame)
    
    # cv2.imshow('Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()